In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
file_path = "weatherAUS.csv"
df = pd.read_csv(file_path)



In [6]:
import pandas as pd

# Load dataset
df = pd.read_csv("weatherAUS.csv")

# Count rows with at least one NaN value
rows_with_na = df.isna().any(axis=1).sum()

print(f"Number of rows containing at least one NaN value: {rows_with_na}")


Number of rows containing at least one NaN value: 59875


In [7]:
# Drop rows with missing values
df_cleaned = df.dropna()

# Drop columns with more than 50% missing values
df_cleaned = df.dropna(thresh=len(df) * 0.5, axis=1)


In [8]:
df.shape

(258620, 19)

In [9]:
df.head()

,Date,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,01-12-2008,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,0.0,No
1,02-12-2008,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,0.0,No
2,03-12-2008,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,0.0,No
3,04-12-2008,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,1.0,No
4,05-12-2008,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,0.2,No


In [10]:
df.shape

(258620, 19)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258620 entries, 0 to 258619
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           258620 non-null  object 
 1   MinTemp        254029 non-null  float64
 2   MaxTemp        254092 non-null  float64
 3   Rainfall       250676 non-null  float64
 4   WindGustDir    239484 non-null  object 
 5   WindGustSpeed  239678 non-null  float64
 6   WindDir9am     238427 non-null  object 
 7   WindDir3pm     247155 non-null  object 
 8   WindSpeed9am   253199 non-null  float64
 9   WindSpeed3pm   248800 non-null  float64
 10  Humidity9am    252555 non-null  float64
 11  Humidity3pm    247670 non-null  float64
 12  Pressure9am    230100 non-null  float64
 13  Pressure3pm    230122 non-null  float64
 14  Temp9am        254031 non-null  float64
 15  Temp3pm        249034 non-null  float64
 16  RainToday      250676 non-null  object 
 17  RISK_MM        250674 non-nul

In [ ]:
# Drop rows where target variable "RISK_MM" is missing
df = df.dropna(subset=["RISK_MM"])

# Fill missing numerical values with the median
num_cols = df.select_dtypes(include=["float64"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Encode categorical variables using Label Encoding
cat_cols = ["WindGustDir", "WindDir9am", "WindDir3pm", "RainToday"]
label_encoders = {col: LabelEncoder() for col in cat_cols}

for col in cat_cols:
    df[col] = label_encoders[col].fit_transform(df[col].astype(str))

# Convert Date to datetime and extract features
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df.drop(columns=["Date"], inplace=True)

# Define features and target variable
X = df.drop(columns=["RISK_MM"])
y = df["RISK_MM"]

# Remove target variable from numerical columns before scaling
num_cols = num_cols.drop("RISK_MM")

# Scale numerical features
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")
